Initializes variant of **MarianMT** model (English -> Tamil)

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
# 308 M
import os
from dotenv import load_dotenv

load_dotenv()

model_name = "Helsinki-NLP/opus-mt-en-dra"
hf_token = os.getenv("HF_TOKEN") # replace with hf token

tokenizer = MarianTokenizer.from_pretrained(model_name, use_auth_token=hf_token)
model = MarianMTModel.from_pretrained(model_name, use_auth_token=hf_token)


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1908: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


source.spm:   0%|          | 0.00/818k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.17M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:4697: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Sample test of non-quantized model

In [ ]:
# Prepare the input text with the target language token
english_text = "hello"
input_text = f">>tam<< {english_text}"

# Tokenize and translate
inputs = tokenizer(input_text, return_tensors="pt")
translated = model.generate(**inputs)
tamil_text = tokenizer.decode(translated[0], skip_special_tokens=True)

print("Tamil:", tamil_text)

Tamil: ஹலோ


Turns different layers in transformer to use quantized int 8 instead of floating point 32, since it's more memory efficient

In [ ]:
import torch

# Apply dynamic quantization to Linear, LSTM, and GRU layers
quantized_model = torch.quantization.quantize_dynamic(
    model,
    {torch.nn.Linear, torch.nn.LSTM, torch.nn.GRU},  # quantize Linear, LSTM, GRU
    dtype=torch.qint8  # 8-bit integers
)

Sample english and tamil dataset for testing the model's BLEU score (Metric for seeing our accuracy between language conversions)

In [ ]:
import kagglehub
import os

# Download dataset
path = kagglehub.dataset_download("hemanthkumar21/englist-tamil-parallel-sent")
print("Dataset downloaded to:", path)

# List files inside
for root, dirs, files in os.walk(path):
    for f in files:
        print(os.path.join(root, f))

import pandas as pd

file_path = os.path.join(path, "en-ta", "general_en_ta 87k.csv")
df = pd.read_csv(file_path)

print(df.tail())


100%|██████████| 187M/187M [00:01<00:00, 153MB/s]

Extracting files...


Dataset downloaded to: /root/.cache/kagglehub/datasets/hemanthkumar21/englist-tamil-parallel-sent/versions/1
/root/.cache/kagglehub/datasets/hemanthkumar21/englist-tamil-parallel-sent/versions/1/en-ta/corpus.bcn.dev 1k.csv
/root/.cache/kagglehub/datasets/hemanthkumar21/englist-tamil-parallel-sent/versions/1/en-ta/parallel 8k gloss.xlsx
/root/.cache/kagglehub/datasets/hemanthkumar21/englist-tamil-parallel-sent/versions/1/en-ta/corpus.bcn.train 166k.csv
/root/.cache/kagglehub/datasets/hemanthkumar21/englist-tamil-parallel-sent/versions/1/en-ta/ij_2021_v1 816k .csv
/root/.cache/kagglehub/datasets/hemanthkumar21/englist-tamil-parallel-sent/versions/1/en-ta/en-ta 118k.csv
/root/.cache/kagglehub/datasets/hemanthkumar21/englist-tamil-parallel-sent/versions/1/en-ta/ie_tech 18k.csv
/root/.cache/kagglehub/datasets/hemanthkumar21/englist-tamil-parallel-sent/versions/1/en-ta/pmindia.v1.ta-en 39k.csv
/root/.cache/kagglehub/datasets/hemanthkumar21/englist-tamil-parallel-sent/versions/1/en-ta/general

       Unnamed: 0                                        0  \
87523       87523  I prefer going on foot to going by bus.   
87524       87524  I ran into Mary at the party last week.   
87525       87525  I ran into a friend of mine on the bus.   
87526       87526  I ran into the classroom out of breath.   
87527       87527  I read this book a couple of years ago.   

                                                       1  
87523  பேருந்தில் செல்வதை விட, நடந்தே செல்வதை நான் வி...  
87524      நான் கடந்த வாரம் விருந்தில் மேரிக்கு ஓடினேன்.  
87525                  பேருந்தில் என் நண்பரிடம் ஓடினேன்.  
87526             மூச்சுத் திணறி வகுப்பறைக்குள் ஓடினேன்.  
87527  இந்த புத்தகத்தை நான் இரண்டு வருடங்களுக்கு முன்...  


Just renaming the columns for readability

In [ ]:
df = df.rename(columns={'0': "english", '1': "tamil"})


# Check the result
print(df.tail())


Index(['Unnamed: 0', '0', '1'], dtype='object')
       Unnamed: 0                                  english  \
87523       87523  I prefer going on foot to going by bus.   
87524       87524  I ran into Mary at the party last week.   
87525       87525  I ran into a friend of mine on the bus.   
87526       87526  I ran into the classroom out of breath.   
87527       87527  I read this book a couple of years ago.   

                                                   tamil  
87523  பேருந்தில் செல்வதை விட, நடந்தே செல்வதை நான் வி...  
87524      நான் கடந்த வாரம் விருந்தில் மேரிக்கு ஓடினேன்.  
87525                  பேருந்தில் என் நண்பரிடம் ஓடினேன்.  
87526             மூச்சுத் திணறி வகுப்பறைக்குள் ஓடினேன்.  
87527  இந்த புத்தகத்தை நான் இரண்டு வருடங்களுக்கு முன்...  


Randomizing the dataset

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming your dataframe has 'english' and 'tamil' columns
# Optional: shuffle the dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

Training samples: 70022
Testing samples: 17506


Calculates bleu score with quantized and optimized model

In [ ]:
import torch
from tqdm import tqdm
import sacrebleu

def evaluate_bleu_beam(model, tokenizer, test_df, src_col="english", tgt_col="tamil", num_beams=4, max_samples=None):
    refs = []
    hyps = []

    if max_samples:
        test_df = test_df.iloc[:max_samples]

    for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Evaluating BLEU"):
        src_text = row[src_col]
        tgt_text = row[tgt_col]

        inputs = tokenizer(src_text, return_tensors="pt")
        with torch.no_grad():
            output_tokens = model.generate(
                **inputs,
                num_beams=num_beams,
                max_length=50,
                early_stopping=True
            )
        translated = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

        refs.append([tgt_text])
        hyps.append(translated)

    bleu = sacrebleu.corpus_bleu(hyps, list(zip(*refs)))
    return bleu.score

# Example usage
bleu_score = evaluate_bleu_beam(quantized_model, tokenizer, df, num_beams=4, max_samples=200)
print(f"BLEU score with quantized_model: {bleu_score:.2f}")


Evaluating BLEU: 100%|██████████| 200/200 [00:38<00:00,  5.16it/s]


BLEU score with beam search: 0.86


Save quantized model to google drive

In [ ]:
from google.colab import drive
import torch
import os

# 1️⃣ Mount Google Drive
drive.mount('/content/drive')

# 2️⃣ Create the LangIQ folder if it doesn't exist
save_folder = "/content/drive/MyDrive/LangIQ"
os.makedirs(save_folder, exist_ok=True)

# 3️⃣ Save the entire quantized model
torch.save(quantized_model, os.path.join(save_folder, "quantized_marianmt_model.pth"))

# OR save only the state_dict (smaller, more flexible)
torch.save(quantized_model.state_dict(), os.path.join(save_folder, "quantized_marianmt_state_dict.pth"))

print(f"Model saved to {save_folder}")


Mounted at /content/drive
Model saved to /content/drive/MyDrive/LangIQ
